# Retrieval augmented chat - Mistral-7B-Instruct

This notebook is the primary demonstration of the project with the baseline model. Here we'll bring up the baseline model and vector database and start asking questions both with and without the vector database.

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

## Initialize some variables

In [2]:
collection_name = "Corpus"
model_dir = "mistralai/Mistral-7B-Instruct-v0.1"
device_map = {"": 0}
device = "cuda"
database_top_n_results = 2

## Load shared code
This file defines the `ChatModel` and `Retrieval` classes used below.

In [3]:
%run shared_code.ipynb

## Load the model

In [4]:
language_model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

tokenizer = AutoTokenizer.from_pretrained(model_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Load the model into the ChatModel class from `shared_code.ipynb`

In [5]:
chat_model = ChatModel(language_model, tokenizer)

## Trying the model without access to the vector database.

In [6]:
chat_model.basic_chat("How do I loop a GIF?")

To loop a GIF, you can use HTML5 video tag and set the `loop` attribute to `true`. Here's an example code snippet:
```html
<video src="your-gif-url" loop></video>
```
You can also use CSS to loop an animated GIF by setting the `animation-name` property to the name of the GIF animation and the `animation-iteration-count` property to a number greater than 1. Here's an example code snippet:
```css
.your-gif-class {
  animation: your-animation-name your-animation-duration ease-in-out infinite;
}
```
Note that looping an animated GIF using CSS requires the GIF to be inline in the HTML code and the CSS class to be applied to the GIF element.


## Load the collection into the RetrievalAugmentedChat class from `shared_code.ipynb`

In [7]:
rac = RetrievalAugmentedChat("db/", collection_name, database_top_n_results, chat_model)

## Run retrieval augmented chat
Notice that the responses have switched from a general HTML/JavaScript context to include the document contents which provide examples of encoding the gif with ImageMagick so that it loops by default.

In [8]:
rac.markdown_chat("How do I loop a GIF?")

To loop a GIF, you can use the ImageMagick command line tool. Here are the steps you can follow:

1. Open a terminal window.
2. Navigate to the directory where your GIF file is saved.
3. Run the following command:
```css
convert image.gif -loop 0 image_loop.gif
```
This will create a new file called `image_loop.gif` that will loop indefinitely. The `-loop 0` option tells ImageMagick to loop the animation forever.

If you want to specify a specific number of times you want the GIF to loop, you can use the `-delay` option followed by the number of milliseconds between each frame of the GIF. For example, to loop the GIF 5 times with a delay of 100 milliseconds between each frame, you can use the following command:
```css
convert image.gif -delay 100 -loop 5 image_loop.gif
```
The `-delay` option specifies the time between each frame of the GIF, in milliseconds. A value of 0 means that the frames will be displayed as fast as possible.

Once the command finishes running, you will have a new file called `image_loop.gif` that will loop indefinitely. You can view this file in any media player that supports GIFs.

 **Reference documents:** 

* [corpus/imagemagick/set-a-gif-to-loop.md](corpus/imagemagick/set-a-gif-to-loop.md) distance: 0.97
* [corpus/imagemagick/compress-animated-gif.md](corpus/imagemagick/compress-animated-gif.md) distance: 1.28

**Inference time in seconds 6.3233**


In [9]:
rac.markdown_chat("Can I use npx with GitHub actions?")

Yes, you can use `npx` with GitHub Actions. However, by default, the `npx` command is not cached in the Actions cache.

To use `npx` with GitHub Actions, you need to create a cache key that includes the dependencies required by `npx`. You can do this by setting the `cache-dependency-path` parameter in the `actions/setup-node` action. This action sets up the Node.js runtime and caches the dependencies required by your project.

Here is an example of how to use `npx` with GitHub Actions:
```yaml
- name: Set up Node.js
  uses: actions/setup-node@v2
  with:
    node-version: '14'
    cache: 'npm'
    cache-dependency-path: '**/*'

- name: Install dependencies
  run: npm install

- name: Run npx command
  run: npx my-command
```
In this example, the `actions/setup-node` action is used to set up Node.js and cache the dependencies required by your project. The `cache-dependency-path` parameter is set to `'**/*'`, which means that all files in your project are included in the cache key.

The `install` command is then used to install the dependencies required by your project.

Finally, the `npx` command is used to run the `my-command` command. Since the dependencies are cached, the first time the `npx` command is run, it will download the dependencies from the cache instead of re-installing them from npm.

It's important to note that any change to the cache key, such as adding or removing files, will invalidate the cache and cause the cache to be rebuilt. This can be done manually by clicking the "Flush cache" button on the Actions page or by using the `actions/cache@v2` action in your workflow.

 **Reference documents:** 

* [corpus/github-actions/npm-cache-with-npx-no-package.md](corpus/github-actions/npm-cache-with-npx-no-package.md) distance: 0.88
* [corpus/github-actions/attach-generated-file-to-release.md](corpus/github-actions/attach-generated-file-to-release.md) distance: 1.11

**Inference time in seconds 9.0233**
